# Sistema 3

## Simulación de sistemas de Polímeros
En este problema pasamos a un sistema con mayor complejidad. Simularemos dos péptidos de <tt>N_Beads</tt> monómeros con distinta carga. ESPResSO tiene disponible un comando para preparar posiciones aleatorias para definir las partículas de los polímeros. Además introduciremos interacciones enlazantes a partir de un potencial armónico e interacciones electrostáticas en nuestro sistema cargado.

En este caso, nos hace falta importar nuevas funcionalidades del módulo <tt>espressomd</tt>.

In [1]:
import espressomd
from espressomd import polymer
from espressomd.interactions import HarmonicBond
import espressomd.electrostatics
import numpy as np
import tipus_particules as tp

In [2]:
# Dimensiones del sistema
system=espressomd.System(box_l=[10,10,10])   # Inicializamos el sistema. Caja de 10x10x10 nm3

# Parámetros del sistema
temperatura = 300
R           = 8.314472
RT          = temperatura * R

# Parámetros de los polímeros
N_Beads=10
dist_Beads=0.5   # nm
Semilla = 20

In [3]:
#Partículas del sistema

#Código numérico de los tipos de partículas.
type_A = 0     
type_Q = 2

#Parámetros de carga
charges = {}
charges[type_A] = 0
charges[type_Q] = -1.

#Parámetros de masa en kg/mol
mass = {}
mass[type_A] = 0.050   # En kg/mol
mass[type_Q] = 0.050   # En kg/mol

# 
D_coeff = {}  # En nm2/ns
D_coeff[type_A] = 1.0      
D_coeff[type_Q] = 1.0      


# Fricciones. Isotrópicas 

Fric={}
Fric[type_A]=[RT/D_coeff[type_A]]*3
Fric[type_Q]=[RT/D_coeff[type_Q]]*3

In [4]:
#317 kcal/(mol A2)
317000/4.184*100

7576481.835564053

In [5]:
# Generamos el polímero

#Coordenadas aleatorias iniciales
polymer_xyz = polymer.positions(n_polymers=2,
                             beads_per_chain=N_Beads,
                             bond_length=dist_Beads, seed=Semilla)

# MODIFICAR AQUI LA POSICION DE LOS POLIMEROS
#polymer_xyz[1]=...

#Parámetros del enlace 
k_bond_CC=7.6E6        #     en J/(mol nm2) = 317 kcal/(molA2)
hb = HarmonicBond(k=k_bond_CC, r_0=dist_Beads)   # Unidades J/molnm2 i r_0: nm
system.bonded_inter.add(hb)


#POLIMERO NEUTRO

for i_bead in range(N_Beads):
    id = len(system.part)
    system.part.add(id=id, pos=polymer_xyz[0][i_bead], type=type_A, q=charges[type_A]
                        ,mass=mass[type_A],gamma=Fric[type_A])    
    # Definir enlaces del bead (id) con el (id-1)
    if i_bead > 0:
        system.part[id].add_bond((hb, id - 1))
    

#POLIMERO CARGADO

for i_bead in range(N_Beads):
    id = len(system.part)
    system.part.add(id=id, pos=polymer_xyz[1][i_bead], type=type_Q, q=charges[type_Q]
                        ,mass=mass[type_Q],gamma=Fric[type_Q])    
    # Definir enlaces del bead (id) con el (id-1)
    if i_bead > 0:
        system.part[id].add_bond((hb, id - 1))    

La función <tt>polymer.positions</tt> permite construir nuestro polímero en posiciones aleatorias y adecuadas. Los argumentos requeridos nos permite configurar el número de polímeros, los beads por cadena de polímero y la distancia entre beads. Será necesario proporcionarle una semilla para generar las posiciones mediante un algoritmo generador de números pseudo-aleatorios. 

En este caso, asignamos partículas <tt>type_A</tt> (sin carga) a las posiciones aleatorias para el primer polímero <tt>polymer_xyz[0]</tt> mediante la función <tt>system.part.add</tt>. Repetimos el proceso con partículas <tt>type_Q</tt> (carga negativa) para el polímero  <tt>polymer_xyz[1]</tt>. 

Adicionalmente definimos el potencial armónico para los enlaces entre beads mediante la función <tt>HarmonicBond</tt>, el cual requiere que proporcionemos una constante de fuerza <tt>k</tt> y la distancia de equilibrio <tt>r0</tt>. Fíjate cómo añadimos el enlace <tt>hb</tt> (HarmonicBond) en la partícula <tt>id</tt> con la partícula <tt>id - 1</tt> mediante la función <tt>add_bond</tt>. De este modo añadimos las partículas y sus interacciones enlazantes iterativamente.

In [9]:
# Interacciones
    
lj_epsilon=5000.
lj_sigma=0.5
lj_cutoff=3.
lj_shift=0.
r_cut_DH=system.box_l[0]*0.4

system.non_bonded_inter[type_A, type_A].lennard_jones.set_params(
                epsilon=lj_epsilon, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    
    
system.non_bonded_inter[type_Q, type_A].lennard_jones.set_params(
                epsilon=lj_epsilon, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    

system.non_bonded_inter[type_Q, type_Q].lennard_jones.set_params(
                epsilon=lj_epsilon/100, sigma=lj_sigma, cutoff=lj_cutoff, shift=lj_shift)    
    
    
# Variables Medi
kappa_gas=138072.   # en J/(mol nm)   Prefactor de Coulomb en fase gas

# Constante dieléctrica del disolvente. Ej. Agua
die_solvent=80

prefactor_val=kappa_gas/die_solvent    
kappa_DH=0   # Si 0 Sin fuerza iónica. Para kappa_DH=0 corresponde a Coulombico puro

DHsolv = espressomd.electrostatics.DH(prefactor=prefactor_val,kappa=kappa_DH,r_cut=r_cut_DH,check_neutrality=False)
system.actors.add(DHsolv)

En este caso, añadimos las interacciones electrostáticas usando la teoría de Debye-Hückel para incluir la desviación por el efecto del apantallamiento respecto alcomportamento ideal en la solución con electrolitos. Con la función <tt>espressomd.electrostatics.DH</tt> definimos el potencial, que requiere de la inversa de la longitud de apantallamiento de Debye $k$ y un <tt>prefactor</tt>. Para que la interacción elestrotática sea efectiva durante la integración, debe introducirse en la lista de actores activos mediante la función <tt>system.actors.add</tt>. 

Una vez definidas las posiciones, partículas e interacciones de nuestro sistema, realizamos una minimización para reducir las posibles tensiones o superposiciones generadas durante la parametrización o definición de posiciones. Entonces definimos el integrador del método de Steepest Descent (SD) para propagar las partículas en distancias cortas paralelas a las fuerzas que actuan sobre ellas.

In [10]:
# Minimización energética
max_displacement = 0.1 * lj_sigma
damping          = 30
Emin_per_step    = 100000

system.time_step =  0.000003    # ns
system.cell_system.skin =  0.3  

system.integrator.set_steepest_descent(f_max=0, gamma=damping, max_displacement=max_displacement)
system.integrator.run(Emin_per_step)

Para realizar la minimización con SD necesitamos una <tt>f_max</tt> como criterio de convergencia para detener la minimización una vez la fuerza máxima de las partículas está situada por debajo de este umbral. La energía se relaja según <tt>gamma</tt> mientras que el máximo desplazamiento permitido de las coordenadas por paso se define con <tt>max_displacement</tt>.

In [11]:
# Activamos el Modelo Velocity-Verlet
system.integrator.set_vv()

# Activamos el termostáto
system.thermostat.set_langevin(kT=RT, gamma=1 , seed=Semilla)

In [12]:
niter=100
pasos=1000

tp.save_vxyz(system,'Trajectory3.xyz','w',aplicar_PBC=False)


for it in range(pasos):
    system.integrator.run(niter)                                    # Realiza niter de dinámica
    tp.save_vxyz(system,'Trajectory3.xyz','a',aplicar_PBC=False)    # Save coordenada

Una vez terminada la simulación, visualízala en el VMD. Fíjate en el comportamiento de cada polímero según la carga de las partículas. ¿Qué observas?

# Práctica

- Realiza otras simulaciones cambiando la constante dieléctrica del disolvente, variable die_solvent per 1, 20 y compara con el valor del agua de 80.

- Genera un polímero cargado positivamente y otro negativamente

- Estudia las dinámicas con polímeros de diferente medida. Si aumentas mucho el número de monómeros el tiempo de simulación incrementará excesivamente.

